In [2]:
import os
import feedparser
import requests
import subprocess
import assemblyai as aai
import threading

## Configuring Variables


In [23]:
# Podcast RSS Feed URLs
nasil_olunur_url = "https://feeds.simplecast.com/TRRbx_5C"
ybyk_url = "https://feeds.simplecast.com/leyT8Ibc"

# File Directory Variables
download_dir = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/episodes_nasil_olunur"
transcription_folder = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes"

# Track Downloaded Episodes
downloaded_episodes = set()


## Downloading Podcast Episodes

In [18]:
def download_and_compress(title, url):
    """
    Downloads a compressed mp3 version of a podcast episode
    :param title: Episode title
    :param url: The specific episode url from RSS feed
    :return: null
    """

    # clear the filename
    title = title.replace("- ", "-").replace(" ", "_").replace("/", "-")
    filename = os.path.join(download_dir, f"{title}.mp3")
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    ffmpeg_command = [
        "ffmpeg", "-i", url,                # Input URL
        "-b:a", "64k",                      # Set audio bitrate
        "-f", "mp3",                        # Output format
        "-y",                               # Overwrite existing file
        "-loglevel", "error",               # Suppress all but error output
        filename                            # Output file path
    ]
    try:
        subprocess.run(ffmpeg_command, check=True)
        print(f"Downloaded and compressed: {title}")
    except subprocess.CalledProcessError:
        print(f"Failed to download and compress: {title}")

In [50]:
def check_new_episodes(url, begin,end):
    """
    Checks RSS feed for new episodes
    :param url: Podcast's RSS feed url
    :param begin: start checking from this episode no
    :param end: end checking at this episode no
    :return: none
    """
    feed = feedparser.parse(url)

    for episode in feed.entries[begin:end]:
        title = episode["title"]
        audio_url = episode.enclosures[0].href
        print(f"Title: {title} | Audio Url: {audio_url}")
        if title not in downloaded_episodes:
            downloaded_episodes.add(title)
            download_and_compress(title, audio_url)

In [51]:
start_transcription_from = 158
e = len(feed.entries) - start_transcription_from
s = e - 15 # how many more episodes to transcribe

check_new_episodes(nasil_olunur_url, s, e)

Title: 173- Nurcan Akad | Audio Url: https://cdn.simplecast.com/audio/41de0602-25fb-46f9-8041-964fa58ffcaa/episodes/5b9cd505-c949-45cc-a237-6ba648348e60/audio/b6e09549-6534-42ab-832d-e450347a0bbc/default_tc.mp3?aid=rss_feed&feed=TRRbx_5C
Downloaded and compressed: 173-Nurcan_Akad
Title: 172- Yankı Yazgan | Audio Url: https://cdn.simplecast.com/audio/41de0602-25fb-46f9-8041-964fa58ffcaa/episodes/46ac5508-c001-42aa-b717-cc732e6081a6/audio/5e2a8648-83b0-441b-8d33-3b1599f34ae3/default_tc.mp3?aid=rss_feed&feed=TRRbx_5C
Downloaded and compressed: 172-Yankı_Yazgan
Title: 171- Okan Bayülgen | Audio Url: https://cdn.simplecast.com/audio/41de0602-25fb-46f9-8041-964fa58ffcaa/episodes/36d12788-245d-420d-b342-4f21b270d90e/audio/ee023f3e-87d6-41b6-8ec4-d87047505c08/default_tc.mp3?aid=rss_feed&feed=TRRbx_5C
Downloaded and compressed: 171-Okan_Bayülgen
Title: 170- Serra Yılmaz | Audio Url: https://cdn.simplecast.com/audio/41de0602-25fb-46f9-8041-964fa58ffcaa/episodes/7815f677-f364-4352-8935-642b4d951e

## Transcription with Assembly AI

In [ ]:
aai.settings.api_key = "ASSEMBLY_API_KEY"


config = aai.TranscriptionConfig(
    speaker_labels=True,
    speakers_expected=2,
    language_code="tr",
    entity_detection=True #entity detection ON
)

# Simdilik sadece Nasil Olunur icin yaptim
config.set_custom_spelling(
  {
    "Nilay": ["Nile"],
  }
)

episode_dict = {}

In [ ]:
print(episode_dict)

In [39]:
def transcribeAudio(audio_file):
    """
    Transcribes single audio file (mp3)
    :param audio_file: audio file to be transcripted
    """

    ep_no = audio_file[0:3]
    guest = audio_file[4:-4]

    transcriber = aai.Transcriber(config=config)
    transcript = transcriber.transcribe(os.path.join(download_dir, audio_file))

    episode_dict[audio_file] = [audio_file, ep_no, guest, transcript.id]

In [66]:
# Deneme - Audio URL ile transcription oluyor mu?
audio_url = "https://cdn.simplecast.com/audio/41de0602-25fb-46f9-8041-964fa58ffcaa/episodes/c9cc91a2-fe8c-4594-85ae-f0579007c11c/audio/e4b52022-3aa2-4048-a98d-213e3ace53b0/default_tc.mp3?aid=rss_feed&feed=TRRbx_5C"

def transcribeAudioFromURL(audio_url):
    transcriber = aai.Transcriber(config=config)
    transcript = transcriber.transcribe(audio_url)

    return transcript.text[0:1000]

# transcribeAudioFromURL(audio_url)

id = "7f31f327-581c-40d7-b95c-2cb99669aceb"
transcript = aai.Transcript.get_by_id(id)

print(transcript.text[0:1000])



Merhabalar, ben Nilay Yörnek. Yeni bir şeyleri öğrenmeye merakla herkesin ilgisinin çekeceğini düşündüğüm... ...Nasıl Olunur adlı podcast serimi hoş geldiniz. Bu seride mikrofonu mesleklerini ustalıkla icra ettiğini düşündüğüm insanlara yöneldim. Onlara aynı soruyu soruyorum. Nasıl olunur? Bu bölümde konuğum bir bağlama üstadı... ...Türk Halk Müziği sanatçısı, eğitimci Erdal Erzincan. Hoş geldiniz. Hoş buldum. Box Müzesi'nin Anadolu ödülleri çift yıllarda kurumlara, tek yıllarda kişilere veriliyor. Yeni'yi Aramak üst başlığıyla verilen 2024 müzik ödülünü de Erdal Erzincan Gezici Bağlama Atölyesi aldı. Tebrik ediyorum. Çok teşekkür ederim. 2019'Dan bu yana ödüllerin verilmesi için baya baya çalıştığına şahit oldum. Hüsamettin Koçer'e de söyledim. Hemen akşam konuşuyoruz. Çok havalıydı ödül töreni dedim hocam. Sizin ödülünüzü Bülent Ortaçgil açıkladı. Ekrem İmamoğlu'ndan aldınız ödülü. Baya havalıydı. Başka ödül törenlerinde hocam dedim bir kişi üç ödül birden veriyor sizde dedim iki kiş

In [59]:
def batchTranscription():
    """
    Transcribes multiple audio files concurrently
    """
    threads = []

    for filename in os.listdir(download_dir):
        if int(filename[0:3]) > 158:
            thread = threading.Thread(target=transcribeAudio, args=(filename,))
            threads.append(thread)
            thread.start()

    for thread in threads:
        thread.join()

    print("All transcriptions are complete.")

In [60]:
batchTranscription()

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpx/_transports/default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/connection.py", line 103, in handle_request
    return self._connection.handle_request(request)
  File "/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/httpcore/_sync/http11.py", line 136, in handle_request
    raise exc
  File "/

KeyboardInterrupt: 

### Retrieve Transcript by ID

In [101]:
def get_transcript():

  for key, value in episode_dict.items():
      transcript_id = value[3]
      transcript = aai.Transcript.get_by_id(transcript_id)
      # print(transcript.text[300:400])


In [62]:
t_id = "cce21de0-04df-4e6d-a1db-e283bcefd88d"
transcript = aai.Transcript.get_by_id(t_id)
print(transcript.text[0:1000])

Merhabalar, ben Nilay Örnek. Yeni bir şeyler öğrenmeye merakla herkesin ilgisini çekeceğini düşündüğüm Nasıl Olunur adlı podcast serimi hoş geldiniz. Bu seride mikrofonu mesleklerini ustalıkla icra ettiğini düşündüğüm insanlara yöneltip onlara aynı soruyu soruyorum. Nasıl olunur? Bu bölümde konuğum, profesyonel bilardo oyuncusu, Türkiye'yi dünyada temsil etmiş, dünya şampiyonu olmuş bir isim Semih Saygıner. Hoş geldiniz. Hoş bulduk. Çok memnunum geldiğiniz için. Teşekkür ederim. Şimdi Semih Bey malum, Türkiye'de Bilardo'nun imajını değiştiren, saygınlığını oturtan, Bilardo'ya ilgiyi arttıran bir isim. Lenk lenk vardır Çinli piyanist hep onun yüzlerce, binlerce kişiyi, çocukları özellikle piyanoya teşvik ettiği söylenir Çin'de. Semih Bey'in de Bilardo'da öyle bir etkisi olduğu malum. Tam bir nasıl olunur? İnsanı biraz açmak istiyorum izniniz olursa. Tabii ki. Zaten programda siz de dinleyeceksiniz, göreceksiniz şahane bir anlatıcı. Hem sadece bilardo oyuncusu, bir sporcu değil, müzisyen

## Transcript -> txt file

In [9]:
def write_text_to_file(transcript, title, guest):
    file_path = "/transcripted_episodes/txt_files"
    guest = guest.replace("_", " ")

    text = ""
    for utterance in transcript.utterances:
        if utterance.speaker == "A":
          speaker = "Nilay Örnek"
        elif utterance.speaker == "B":
          speaker = guest

        text += f"{speaker}: {utterance.text}\n"

    with open(f"{file_path}/{title}.txt", "w") as f:
        f.write(text)

In [10]:
# Create txt Files
for key,value in episode_dict.items():
    title = key[:-4]
    guest = value[2]

    transcript_id = value[3]
    transcript = aai.Transcript.get_by_id(transcript_id)

    write_text_to_file(transcript, title, guest)

NameError: name 'episode_dict' is not defined

## Transcript -> .doc file

In [15]:
from docx import Document

new_transcription_folder = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/episodes_with_timestamps"
def write_to_word(transcript, title, guest, ep_no):
  file_path = os.path.join(new_transcription_folder, f"{title}.docx")
  guest = guest.replace("_", " ")

  doc = Document()
  doc.add_heading(f"Nasıl Olunur {ep_no}. Bölüm - {guest}", level=0)

  def time_conversion(millisec):
      second = (millisec // 1000) % 60
      minute = (millisec // 60000) % 60
      hour = millisec // 3600000

      return f"[{hour:02}:{minute:02}:{second:02}]"

  for utterance in transcript.utterances:
    if utterance.speaker == "A":
      speaker = "Nilay Örnek"
    elif utterance.speaker == "B":
      speaker = guest

    time = time_conversion(utterance.start)

    paragraph = doc.add_paragraph()
    paragraph.add_run(f"{speaker} {time}: ").bold = True
    paragraph.add_run(utterance.text)


  doc.save(file_path)

In [8]:
# Create Word Files
for key,value in episode_dict.items():
    title = key[:-4]
    guest = value[2]
    episode_no = value[1]

    transcript_id = value[3]
    transcript = aai.Transcript.get_by_id(transcript_id)

    write_to_word(transcript, title, guest, episode_no)

In [16]:
# Faruk Pekin Re-do
transcript_id = "9eeac035-a137-46ac-9b01-8501c577cea4"
transcript = aai.Transcript.get_by_id(transcript_id)
title = '155 - Faruk_Pekin'
guest = "Faruk Pekin"
episode_no = "155"
write_to_word(transcript, title, guest, episode_no)

## Playing with LLM Feature Extractions

    ### NER Models

#### Akdeniz27 Bert Base Turkish Cased NER

In [8]:
# Use a pipeline as a high-level helper
# from transformers import pipeline
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
f = open("/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/txt_files/146-Armağan_Çağlayan.txt", "r")
text = f.read()
f.close()

model = AutoModelForTokenClassification.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
tokenizer = AutoTokenizer.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")
results = ner(text)

# print(results)
keywords = {}
for item in results:
    word = item["word"].strip()
    label = item["entity_group"]
    keywords[word] = label
    print(item)
#
# for key, val in keywords.items():
#     print(f"{key} | {val}")

# print(keywords)

/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/anaconda3/envs/pdtxt/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'entity_group': 'PER', 'score': np.float32(0.9898243), 'word': 'Nilay Örnek', 'start': 0, 'end': 11}
{'entity_group': 'PER', 'score': np.float32(0.98704255), 'word': 'Nilay Örnek', 'start': 29, 'end': 40}
{'entity_group': 'ORG', 'score': np.float32(0.4778735), 'word': 'YouTube', 'start': 356, 'end': 363}
{'entity_group': 'PER', 'score': np.float32(0.98999584), 'word': 'Armağan Çağlayan', 'start': 374, 'end': 390}
{'entity_group': 'PER', 'score': np.float32(0.98671687), 'word': 'Armağan Çağlayan', 'start': 406, 'end': 425}
{'entity_group': 'PER', 'score': np.float32(0.99024796), 'word': 'Nilay Örnek', 'start': 447, 'end': 458}
{'entity_group': 'LOC', 'score': np.float32(0.9135723), 'word': 'Hereke', 'start': 1122, 'end': 1128}
{'entity_group': 'PER', 'score': np.float32(0.9871603), 'word': 'Armağan Çağlayan', 'start': 1402, 'end': 1421}
{'entity_group': 'LOC', 'score': np.float32(0.99868804), 'word': 'Türkiye', 'start': 1433, 'end': 1440}
{'entity_group': 'ORG', 'score': np.float

#### DBMDZ Bert Base Turkish Cased

In [34]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

f = open("/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/txt_files/146-Armağan_Çağlayan.txt", "r")
text = f.read()
f.close()

model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner=pipeline('ner', model=model, tokenizer=tokenizer)
results = ner(text)

for item in results:
    print(item)

{'entity': 'B-PER', 'score': np.float32(0.9949898), 'index': 1, 'word': 'Nil', 'start': 0, 'end': 3}
{'entity': 'I-PER', 'score': np.float32(0.98402554), 'index': 2, 'word': '##ay', 'start': 3, 'end': 5}
{'entity': 'I-PER', 'score': np.float32(0.99821776), 'index': 3, 'word': 'Örnek', 'start': 6, 'end': 11}
{'entity': 'B-PER', 'score': np.float32(0.9814542), 'index': 8, 'word': 'Nil', 'start': 29, 'end': 32}
{'entity': 'I-PER', 'score': np.float32(0.9374026), 'index': 9, 'word': '##ay', 'start': 32, 'end': 34}
{'entity': 'I-PER', 'score': np.float32(0.9887221), 'index': 10, 'word': 'Örnek', 'start': 35, 'end': 40}
{'entity': 'I-ORG', 'score': np.float32(0.5537669), 'index': 26, 'word': '.', 'start': 121, 'end': 122}
{'entity': 'I-ORG', 'score': np.float32(0.6981019), 'index': 27, 'word': '.', 'start': 122, 'end': 123}
{'entity': 'I-ORG', 'score': np.float32(0.6109145), 'index': 29, 'word': 'Ol', 'start': 129, 'end': 131}
{'entity': 'I-ORG', 'score': np.float32(0.66498953), 'index': 30,

#### GliNER -- tırt çıktı


In [2]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_multi")

f = open("/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/txt_files/146-Armağan_Çağlayan.txt", "r")
longtext = f.read()
f.close()

labels = ["isim", "mekan", "organizasyon", "konu"]

entities = model.predict_entities(longtext, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


Fetching 4 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 65536.00it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Nilay Örnek => isim
Nilay Örnek => isim
YouTube => organizasyon
Armağan Çağlayan => isim
Nilay Örnek => isim
Youtube => organizasyon
kasabalılık => konu
Sultanahmet => mekan
yasa => konu
hukukla => konu
mahkemelerle => konu
Merhaba Kıraathanesi => mekan
Merhaba Durağı => organizasyon
Düzce => mekan
Nilay Örnek => isim


# To-Do Items

YBYK Fikirleri
* yemek adi filtrele
* semt filtrele
* lokanta filtrele
* insan filtrele